Design and implement a data structure for Least Recently Used (LRU) cache. It should support the following 

operations: get and set.

get(key) - Get the value (will always be positive) of the key if the key exists in the cache, otherwise return -1.

set(key, value) - Set or insert the value if the key is not already present. When the cache reached its capacity, it should invalidate the least recently used item before inserting a new item. Finally, you need to return the data from each get

Example1

Input:

    LRUCache(2)
    set(2, 1)
    set(1, 1)
    get(2)
    set(4, 1)
    get(1)
    get(2)

Output: [1,-1,1]

Explanation：

    cache cap is 2，set(2,1)，set(1, 1)，get(2) and return 1，set(4,1) and delete (1,1)，because （1,1）is the least use，get(1) and return -1，get(2) and return 1.
    
Example 2:

Input：

    LRUCache(1)
    set(2, 1)
    get(2)
    set(3, 2)
    get(2)
    get(3)

Output：[1,-1,2]

Explanation：

    cache cap is 1，set(2,1)，get(2) and return 1，set(3,2) and delete (2,1)，get(2) and return -1，get(3) and return 2.
    
https://www.lintcode.com/problem/lru-cache/description

https://leetcode.com/problems/lru-cache/

# Solution

## Solution 1

In [ ]:
# OrderedDict
class LRUCache:

    def __init__(self, capacity: int):
        self.capacity = capacity
        self.dic = collections.OrderedDict()

        
    def get(self, key: int) -> int:
        if key not in self.dic:
            return -1
        value = self.dic.pop(key)
        self.dic[key] = value
        return value
    
    
    def put(self, key: int, value: int) -> None:
        if key in self.dic:
            self.dic.pop(key)
        else:
            if self.capacity > 0:
                self.capacity -= 1
            else:   ### when self.dic is full
                self.dic.popitem(last=False)   ### memorize the syntax of colletions.OrderedDict()
        self.dic[key] = value


# Your LRUCache object will be instantiated and called as such:
# obj = LRUCache(capacity)
# param_1 = obj.get(key)
# obj.put(key,value)

## Solution 2

# History

In [10]:
class LinkedNode:
    def __init__(self, key=None, val=None, next=None):
        self.key = key
        self.val = val
        self.next = next

class LRUCache:
    """
    @param: capacity: An integer
    """
    def __init__(self, capacity):
        # do intialization if necessary
        self.hash = {} ## store the key of prvious node for deleting linked list node
        self.head = LinkedNode() ### dummy node
        self.tail = self.head ##### initialize as a same node
        self.capacity = capacity
        
    def addNodeAtTail(self,node):
        self.hash[node.key] = self.tail
        self.tail.next = node
        self.tail = node
        
    def deleteFirstNode(self): ##### delete the head node of linked list (queue)
        ### remember "self.head" is a dummy node, "self.head.next" is the first node in queue
        del self.hash[self.head.next.key]
        self.head.next = self.head.next.next
        self.hash[self.head.next.key] = self.head ### the previous node of first node is dummy node (head)
        
    def moveNodeToTail(self, previous_node):
        node = previous_node.next
        if node == self.tail:
            return
        previous_node.next = node.next
        if node.next != None:
            self.hash[node.next.key] = previous_node
            node.next = None
        self.addNodeAtTail(node)
        
        
    """
    @param: key: An integer
    @return: An integer
    """
    def get(self, key):
        # write your code here
        if key not in self.hash:
            return -1
        self.moveNodeToTail(self.hash[key])
        return self.hash[key].next.val ### Don't forget self.hash store the previous node of the key

    """
    @param: key: An integer
    @param: value: An integer
    @return: nothing
    """
    def set(self, key, value):
        # write your code here
        if key in self.hash:
            self.moveNodeToTail(self.hash[key])
            self.hash[key].next.val = value ### Update to the new value
        else:
            self.addNodeAtTail(LinkedNode(key, value))
            if len(self.hash) > self.capacity:
                self.deleteFirstNode()

In [17]:
print("=============== Create a LRUCache with capacity = 3 ===============")
a = LRUCache(3)
print("=============== Set key = 1, value = 10 ===============")
a.set(key=1, value=10)
print(a.hash)
print("\n")
print("=============== Set key = 2, value = 20 ===============")
a.set(key=2, value=20)
print(a.hash)
print("\n")
print("=============== Set key = 3, value = 30 ===============")
a.set(key=3, value=30)
print(a.hash)
print("\n")
print("=============== Set key = 2, value = 200 (Update a exist node) ===============")
a.set(key=2, value=200)
print(a.hash)
print("\n")
print("=============== Set key = 4, value = 400 (Add a new node) ===============")
a.set(key=4, value=400)
print(a.hash)
print("\n")
print("=============== Not in the hash ===============")
print(a.get(key=8)) 
print(a.hash)
print("\n")
print("=============== The key is in the hash and update its value ===============")
print(a.get(key=2))
print(a.hash)

=============== Create a LRUCache with capacity = 3 ===============
=============== Set key = 1, value = 10 ===============
{1: <__main__.LinkedNode object at 0x10daeac18>}


=============== Set key = 2, value = 20 ===============
{1: <__main__.LinkedNode object at 0x10daeac18>, 2: <__main__.LinkedNode object at 0x10dafd3c8>}


=============== Set key = 3, value = 30 ===============
{1: <__main__.LinkedNode object at 0x10daeac18>, 2: <__main__.LinkedNode object at 0x10dafd3c8>, 3: <__main__.LinkedNode object at 0x10dafdcf8>}


=============== Set key = 2, value = 200 (Update a exist node) ===============
{1: <__main__.LinkedNode object at 0x10daeac18>, 2: <__main__.LinkedNode object at 0x10daf7198>, 3: <__main__.LinkedNode object at 0x10dafd3c8>}


=============== Set key = 4, value = 400 (Add a new node) ===============
{2: <__main__.LinkedNode object at 0x10daf7198>, 3: <__main__.LinkedNode object at 0x10daeac18>, 4: <__main__.LinkedNode object at 0x10dafdcf8>}


=============== Not 